# 爬取豆瓣书评--基于lxml库

## 一、一本书一个页面上的评论爬取

#### Step1：请求访问网址并拿回网址的信息

In [2]:
##需要安装的库
import sys
import requests
import time
##请求访问网址
url='https://book.douban.com/subject/26954760/comments/hot?p=1'
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:57.0) Gecko/20100101 Firefox/57.0',
}
data=requests.get(url,headers=headers)
data.status_code

200

In [75]:
##查看编码的格式
data.encoding

'utf-8'

#### Step2：网址内容解析

In [76]:
##使用lxml库解析
##lxml库可以通过Xpath路径来定位节点位置
from lxml import etree
selector=etree.HTML(data.text)##selector是一个可以对etree进行选择的选择器

In [191]:
#抽取所有评论区域
comments=selector.xpath('//div[@class="comment"]')

In [194]:
#对评论区域进行解析
users=[]
stars=[]
comment_texts=[]
#遍历
for comment in comments:
    user = comment.xpath(".//h3/span[2]/a/text()")[0]
    star = comment.xpath(".//h3/span[2]/span[1]/@class")
    comment_text = comment.xpath(".//p[@class='comment-content']/span/text()")[0].strip()
    
    users.append(user)
    stars.append(star)
    comment_texts.append(comment_text)
comment_dic = {'user':users , 'srar':stars , 'comments':comment_texts} 

In [192]:
##在循环前看看自己的位置有没有找对  
##有bug  
tep = comments[0]
tep.xpath(".//h3/span[2]/span[1]/@class")[0]
#print(tep.xpath(".//h3/span[2]/a/text()")[0])
#print(tep.xpath(".//p[@class='comment-content']/span/text()")[0].strip())

'user-stars allstar50 rating'

#### Step3：整理数据

In [195]:
##整理数据
import pandas as pd
comments_df = pd.DataFrame(comment_dic)

In [197]:
comments_df[0:6]

,user,srar,comments
0,辣辣周,[user-stars allstar50 rating],听说读完这本书的人，都辞职了。我的领导看到我手里的书，如是说。
1,蘅霜,[user-stars allstar50 rating],斯特里兰克夫人的爱掺杂着虚荣，布兰奇的爱出于自我与报复，唯有阿塔与塔希提岛的海风，才容得下纯...
2,面白饭Θ☉Θ,[user-stars allstar50 rating],怀着感恩和虔诚的心，凌晨六点零六分读完这本书。羡慕里面所有追求梦想，内心坚定的人，甚至羡慕那...
3,A,[user-stars allstar50 rating],然后我就又看了一遍 看到快哭了都！ ps这个版本翻译的极好
4,LIVANTO,[user-stars allstar20 rating],一个傻逼艺术家到处给人添麻烦，以及他无辜朋友们的故事。这个三观太不正了。翻译还不错。
5,洋艾,[user-stars allstar30 rating],或许，六便士是地上的月亮呢


## 二、一本书的书评在多个网页上的爬取

#### Step1：请求网页（多个页面），取回并且合并页面信息

In [16]:
##所以需要设置翻页
url="https://book.douban.com/subject/26954760/comments/"  ##固定的部分
page=('hot?p=')
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:57.0) Gecko/20100101 Firefox/57.0',
}
##循环去遍历
for i in range(1,20):
    if i == 1:
        i=str(i)
        a=(url+page+i+'/')
        r=requests.get(url=a,headers=headers)
        data=r.text
    else:
        i=str(i)
        a=(url+page+i)
        r=requests.get(url=a,headers=headers)
        data2=r.text
        data=data+data2
    time.sleep(3)##设置就是访问页面的时间，若设置时间太短，IP会被封掉

In [17]:
print(a)

https://book.douban.com/subject/26954760/comments/hot?p=19


###### 注：因为每次访问网页的时需要耗费时间的，所以我这里只在循环中合并网页内容，不在循环中做筛选

#### Step2：网址内容解析

In [18]:
##然后重复上面的步骤
selector=etree.HTML(data)
comments=selector.xpath('//div[@class="comment"]')
#对评论区域进行解析
users=[]
stars=[]
comment_texts=[]
#遍历
for comment in comments:
    user = comment.xpath(".//h3/span[2]/a/text()")[0]
    star = comment.xpath(".//h3/span[2]/span[1]/@class")
    comment_text = comment.xpath(".//p[@class='comment-content']/span/text()")[0].strip()
    
    users.append(user)
    stars.append(star)
    comment_texts.append(comment_text)
comment_dic = {'user':users , 'srar':stars , 'comments':comment_texts} 

#### Step3：整理数据

In [19]:
##整理数据
import pandas as pd
comments_df = pd.DataFrame(comment_dic)
comments_df[0:100]

,user,srar,comments
0,辣辣周,[user-stars allstar50 rating],听说读完这本书的人，都辞职了。我的领导看到我手里的书，如是说。
1,蘅霜,[user-stars allstar50 rating],斯特里兰克夫人的爱掺杂着虚荣，布兰奇的爱出于自我与报复，唯有阿塔与塔希提岛的海风，才容得下纯...
2,面白饭Θ☉Θ,[user-stars allstar50 rating],怀着感恩和虔诚的心，凌晨六点零六分读完这本书。羡慕里面所有追求梦想，内心坚定的人，甚至羡慕那...
3,A,[user-stars allstar50 rating],然后我就又看了一遍 看到快哭了都！ ps这个版本翻译的极好
4,LIVANTO,[user-stars allstar20 rating],一个傻逼艺术家到处给人添麻烦，以及他无辜朋友们的故事。这个三观太不正了。翻译还不错。
5,洋艾,[user-stars allstar30 rating],或许，六便士是地上的月亮呢
6,款款,[user-stars allstar50 rating],很庆幸，一读就读到译文最好的一版。会再读一遍原版。
7,肉抖抖,[user-stars allstar40 rating],对女人的态度描写经常让我感到不适，其他很棒
8,喵与Iris,[user-stars allstar50 rating],梦想和现实之间总是有着不可逾越的鸿沟。是妥协着苟且着，还是为了梦想不顾所有，每个人都在纠结着...
9,WiiStone,[user-stars allstar30 rating],"【2018已读59】读完本书的三种人分类。第一种人：""这查理别是个傻逼吧？肯定脑子有问题。""..."


#### Step4：存储数据

In [20]:
##导入进excel
comments_df.to_excel('d:\\comments1.xlsx')

## 三、多本书的书评爬取

###### 思路：点开网页观察
>1、先将每一本书的网址提取出来，作为需要请求的页面                                                                
>2、访问每一个页面后，找到其书评的所在的网页，作为第二步需要请求的页面                                            
>3、进入上一步的页面，找到所有的书评，爬取

#### Step1：进入初始的网页，遍历每一页，将每一页的信息合并

In [10]:
##先取到所有书的链接存入一个list
url="https://www.douban.com/tag/%E5%B0%8F%E8%AF%B4/book"
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:57.0) Gecko/20100101 Firefox/57.0',
}
##因为我们希望取到尽可能多的数目，所以这里就要翻页
##观察页数发现是一个等差数列的变化
import numpy as np
a = np.arange(0,300,15)
page=('?start=')
##循环去遍历
for i in a:
    if i == 0:
        b=url
        r=requests.get(url=b,headers=headers)
        data=r.text
    else:
        i=str(i)
        b=(url+page+i)
        r=requests.get(url=b,headers=headers)
        data2=r.text
        data=data+data2
    time.sleep(1)

In [13]:
print(b)

https://www.douban.com/tag/%E5%B0%8F%E8%AF%B4/book?start=285


#### Step2：解析网页，找到评论页面的链接网址

In [14]:
from lxml import etree
selector1=etree.HTML(data)
print(selector1)

<Element html at 0x1f76e1dc208>


In [15]:
##得到所有的网址
url_sum=selector1.xpath('//dd/a/@href')
print(url_sum)

['https://book.douban.com/subject/26954760/?from=tag_all', 'https://book.douban.com/subject/4913064/?from=tag_all', 'https://book.douban.com/subject/27053166/?from=tag_all', 'https://book.douban.com/subject/1770782/?from=tag_all', 'https://book.douban.com/subject/6518605/?from=tag_all', 'https://book.douban.com/subject/26879778/?from=tag_all', 'https://book.douban.com/subject/26715115/?from=tag_all', 'https://book.douban.com/subject/6082808/?from=tag_all', 'https://book.douban.com/subject/3259440/?from=tag_all', 'https://book.douban.com/subject/1084336/?from=tag_all', 'https://book.douban.com/subject/1082154/?from=tag_all', 'https://book.douban.com/subject/10594787/?from=tag_all', 'https://book.douban.com/subject/25862578/?from=tag_all', 'https://book.douban.com/subject/1008145/?from=tag_all', 'https://book.douban.com/subject/5363767/?from=tag_all', 'https://book.douban.com/subject/2567698/?from=tag_all', 'https://book.douban.com/subject/26878124/?from=tag_all', 'https://book.douban.co

In [21]:
data=str()
##每一个网页检索，需要找到他们的节点，返回他们的网页
for i in url_sum:
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:57.0) Gecko/20100101 Firefox/57.0',
}
    r=requests.get(url=i,headers=headers)
    data3=r.text
    data=data+data3
    time.sleep(1)
   

In [22]:
##找到每一个评论网页
selector2=etree.HTML(data)
url_list=selector2.xpath('//div[@class="mod-hd"]/h2/span[2]/a/@href')   

In [23]:
print(url_list)

['https://book.douban.com/subject/26954760/comments/', 'https://book.douban.com/subject/4913064/comments/', 'https://book.douban.com/subject/27053166/comments/', 'https://book.douban.com/subject/1770782/comments/', 'https://book.douban.com/subject/6518605/comments/', 'https://book.douban.com/subject/26879778/comments/', 'https://book.douban.com/subject/26715115/comments/', 'https://book.douban.com/subject/6082808/comments/', 'https://book.douban.com/subject/3259440/comments/', 'https://book.douban.com/subject/1084336/comments/', 'https://book.douban.com/subject/1082154/comments/', 'https://book.douban.com/subject/10594787/comments/', 'https://book.douban.com/subject/25862578/comments/', 'https://book.douban.com/subject/1008145/comments/', 'https://book.douban.com/subject/5363767/comments/', 'https://book.douban.com/subject/2567698/comments/', 'https://book.douban.com/subject/26878124/comments/', 'https://book.douban.com/subject/1007305/comments/', 'https://book.douban.com/subject/30254

#### Step3：请求访问每一个评论网页，并将所有的信息拼接

In [24]:
##访问每一个评论网页
data2=str()
for i in url_list:
    r=requests.get(url=i,headers=headers)
    data1=r.text
    data2=data2+data1
    time.sleep(1)

#### Step4：解析网页

In [25]:
selector3=etree.HTML(data2)

In [26]:
users=[]
stars=[]
comment_texts=[]
#遍历
comments=selector3.xpath('//div[@class="comment"]')
for comment in comments:
    
    user = comment.xpath(".//h3/span[2]/a/text()")[0]
    star = comment.xpath(".//h3/span[2]/span[1]/@class")
    comment_text = comment.xpath(".//p[@class='comment-content']/span/text()")[0].strip()
    
    users.append(user)
    stars.append(star)
    comment_texts.append(comment_text)
comment_dic = {'user':users , 'srar':stars , 'comments':comment_texts} 

#### Step4：整理数据

###### 注：
>这一部分有一点小问题，因为每一个页面只出现了一次书名，所以所以没有办法和其他的信息一一匹配，所以这里强制处理了一下，让每一个书名都重复了与评论个数相同的次数（20）
>但这个方法没法推广，仍待改进

In [27]:
bookname = selector3.xpath('//*[@id="content"]/h1/text()')
print(bookname)

['月亮与六便士 短评', '活着 短评', '红楼梦 短评', '追风筝的人 短评', '三体全集 短评', '杀死一只知更鸟 短评', '人间失格 短评', '百年孤独 短评', '白夜行 短评', '小王子 短评', '活着 短评', '霍乱时期的爱情 短评', '解忧杂货店 短评', '围城 短评', '三体Ⅲ 短评', '三体 短评', '我的天才女友 短评', '红楼梦 短评', '4 3 2 1 短评', '嫌疑人X的献身 短评', '美丽新世界 短评', '1984 短评', '房思琪的初恋乐园 短评', '杀死一只知更鸟 短评', '平凡的世界（全三部） 短评', '云边有个小卖部 短评', '月亮和六便士 短评', '万历十五年 短评', '三体Ⅱ 短评', '我们仨 短评', '朝花夕拾 短评', '茨威格文集 短评', '飘 短评', '局外人 短评', '撒哈拉的故事 短评', '不能承受的生命之轻 短评', '沉默的大多数 短评', '月亮和六便士 短评', '新名字的故事 短评', '人间失格 短评', '傲慢与偏见 短评', '哈利·波特与魔法石 短评', '许三观卖血记 短评', '挪威的森林 短评', '明朝那些事儿（壹） 短评', '白夜行 短评', '巨人的陨落 短评', '边城 短评', '天才在左 疯子在右 短评', '那不勒斯故事四部曲 短评', '月亮和六便士 短评', '无中生有 短评', '基督山伯爵 短评', '东方快车谋杀案 短评', '恶意 短评', '黄金时代 短评', '动物农场 短评', '马尔多罗之歌 短评', '漫长的告别 短评', '无人生还 短评', '灿烂千阳 短评', '情人 短评', '三国演义（全二册） 短评', '福尔摩斯探案全集（上中下） 短评', '了不起的盖茨比 短评', '呼啸山庄 短评', '夹边沟记事 短评', '城南旧事 短评', '哈利·波特与阿兹卡班的囚徒 短评', '哈利·波特与混血王子 短评', '情书 短评', '鱼王 短评', '狼图腾 短评', '呐喊 短评', '笑傲江湖（全四册） 短评', '明朝那些事儿.第6部,日暮西山 短评', '我与地坛 短评', '海边的卡夫卡 短评', '骆驼祥子 短评', '红玫瑰与白玫瑰 短评', '步履不停 短评', '

In [28]:
##每一页都有20个评论
bookname=np.array(bookname)
bookname=bookname.repeat(20)
print(bookname)

['月亮与六便士 短评' '月亮与六便士 短评' '月亮与六便士 短评' ... '灯塔 短评' '灯塔 短评' '灯塔 短评']


In [29]:
type(bookname)
[bookname]

[array(['月亮与六便士 短评', '月亮与六便士 短评', '月亮与六便士 短评', ..., '灯塔 短评', '灯塔 短评',
        '灯塔 短评'], dtype='<U21')]

In [30]:
##拼接
comments_df = pd.DataFrame(comment_dic)

In [31]:
bookname_df= pd.DataFrame(bookname)

In [33]:
df=bookname_df.join(comments_df)

In [34]:
##导入进excel
df.to_excel('d:\\comments2.xlsx')

## 四、总结
>1、应用lxml库感觉会比BS方便一些，因为它是一个xpath的形式，可以迅速的定位                                         
>2、由于每次访问页面都会耗费时间，所以应该考虑找一群代理，提高速度，避免IP被封                                    
>3、最后的爬取中，每一本书只爬取了一个页面上的评论，可以考虑嵌套循环，找到更多页面的评论（因为速度太慢和书名不好处理，所以没有进行这一步操作）                                                                                     
>4、整体的代码写的都很简单也比较笨，应该有便捷的方法来进行改进